Copyright (c) Rahul Kumar. All rights reserved.

Licensed under the MIT License.

.

# Debug Remote Jobs
This notebook demonstrates how to debug the job remotely like developers typically do on their local machines. Jobs can be interacted with via different training applications including ``JupyterLab``, ``TensorBoard``, ``VSCode`` or by connecting to the job container directly via ``SSH``. Please refer [this link](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-interactive-jobs?tabs=ui#attach-a-debugger-to-a-job) for further information.

## Workspace

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(
    DefaultAzureCredential()
)

## Configure Command job
- The ``JobService`` class is required which has to be used in ``services`` parameter of ``command`` job.
- Pay special attention to the ``command`` parameter of ``command`` job, ``debugpy`` has to be used in order to debug with ``VSCode``.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.entities import JobService

# create the command
job = command(
    code="../scripts",  # local path where the code is stored
    command="python -m debugpy --listen localhost:5678 --wait-for-client main.py --diabetes-csv ${{inputs.diabetes}}",
    inputs={
        "diabetes": Input(
            type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/diabetes.csv",
        )
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-cluster",
    display_name="sklearn-diabetes-example",
    services={
      "My_jupyterlab": JobService(
        job_service_type="JupyterLab",
        nodes="all" # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
      ),
      "My_vscode": JobService(
        job_service_type="VSCode",
        nodes="all"
      ),
      "My_tensorboard": JobService(
        job_service_type="TensorBoard",
        nodes="all",
        properties={
            "logDir": "output/tblogs"  # relative path of Tensorboard logs (same as in your training script)
        }          
      ),
      "My_ssh": JobService(
        job_service_type="SSH",
        sshPublicKeys="<add-public-key>",
        nodes="all",
        properties={
            "sshPublicKeys":"<add-public-key>"
        }    
      ),
    }
    # description,
    # experiment_name
)

## Run the Command

In [ ]:
returned_job = ml_client.create_or_update(job)
returned_job